<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/3_TensorFlow_tf_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

print(tf.__version__, keras.__version__)

In [29]:
# tf.Variable

v = tf.Variable(1213)
print(v)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1213>


In [40]:
# tf.range

tf.range(1) #[0]
tf.range(2) #[0, 1]
tf.range(3) #[0, 1, 2]
tf.range(4) #[0, 1, 2, 3]

start = 3
limit = 18
delta = 3
tf.range(start, limit, delta)  # [3, 6, 9, 12, 15]

start = 3
limit = 1
delta = -0.5
tf.range(start, limit, delta)  # [3, 2.5, 2, 1.5]

limit = 5
tf.range(limit)  # [0, 1, 2, 3, 4]

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4], dtype=int32)>

In [53]:
# tf.range

start = 5
limit = 16
delta = 3
tf.range(start, limit, delta)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 5,  8, 11, 14], dtype=int32)>

In [0]:
# add

@tf.function 
def f(x, y): 
  return x + y 

f(1, 2)

In [0]:
# multiply

@tf.function 
def f(x, y): 
  return x ** 2 + y 
x = tf.constant([2]) 
y = tf.constant([3]) 
f(x, y)

In [11]:
# multiply arrays


@tf.function 
def f(x, y): 
  return x ** 2 + y 
x = tf.constant([2, 3]) 
y = tf.constant([3, -2]) 
f(x, y) 


<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 7], dtype=int32)>

In [0]:
# reduce

@tf.function 
def f(x): 
  if tf.reduce_sum(x) > 0: 
    return x * x 
  else: 
    return -x // 2 
f(tf.constant(-13))

In [0]:
# tf.Tensor and tf.Variable objects:

@tf.function 
def f(): 
  return x ** 2 + y 
x = tf.constant([-2, -3]) 
y = tf.Variable([3, -2]) 
f() 


In [71]:
 # tf.print, tf.Variable and others:

v = tf.Variable(1) 
@tf.function 
def f(x): 
  for i in tf.range(x): 
    v.assign_add(i) 
f(4)
v


<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

In [72]:
#tf.TensorArray:

@tf.function 
def f(x): 
  ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True) 
  for i in range(len(x)): 
    ta = ta.write(i, x[i] + 1) 
  return ta.stack() 
f(tf.constant([1, 2, 3])) 


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 4], dtype=int32)>

In [76]:
# Passing python scalars or lists as arguments to tf.function

@tf.function 
def f(x): 
  return tf.abs(x) 
f1 = f.get_concrete_function(1) 
f2 = f.get_concrete_function(2)  # Slow - builds new graph 
f1 is f2 # False

f1 = f.get_concrete_function(tf.constant(1)) 
f2 = f.get_concrete_function(tf.constant(2))  # Fast - reuses f1 
f1 is f2 # True 


True

In [77]:
# two separate graphs, each specialized to a different shape:

@tf.function 
def f(x): 
  return x + 1 
vector = tf.constant([1.0, 1.0]) 
matrix = tf.constant([[3.0]]) 
f.get_concrete_function(vector) is f.get_concrete_function(matrix) 


False

In [78]:
#  avoid creating multiple graphs when Tensors have dynamic shapes.

@tf.function( 
    input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)]) 
def f(x): 
  return x + 1 
vector = tf.constant([1.0, 1.0]) 
matrix = tf.constant([[3.0]]) 
f.get_concrete_function(vector) is f.get_concrete_function(matrix) 


True

In [0]:
# Variables may only be created once! tf.function only allows creating new tf.Variable objects when it is called for the first time:

class MyModule(tf.Module): 
  def __init__(self): 
    self.v = None 
 
  @tf.function 
  def call(self, x): 
    if self.v is None: 
      self.v = tf.Variable(tf.ones_like(x)) 
    return self.v * x 

